# Simple example for how to load and plot ice data

This script shows how to load and plot sea ice concentration from CICE output, while also indicating how to get around some of the pitfalls and foibles in CICE temporal and spatial gridding.

Requirements: The conda/analysis3 module from /g/data/hh5/public/modules. 

Firstly, load modules:

In [1]:
import intake
catalog = intake.cat.access_nri
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client
from datetime import timedelta
import cf_xarray as cfxr
import cartopy.crs as ccrs

In [2]:
client = Client("tcp://10.6.1.64:8786")
client

<Client: 'tcp://10.6.1.64:8786' processes=1 threads=48, memory=188.56 GiB>

Start a database session:

Load sea ice area (`aice_m`) from the Repeat-Year forcing experiment. (We could, alternatively, try to load ice thickness (`hi_m`) or ice volume (`vicen_m`).) Note that we are just loading the last 10 years here.

Note also the `decode_coords=False` flag. **This gets around some messy issues with the way xarray decides to load CICE grids**:

In [3]:
sic_args = {
    "expt": '1deg_jra55_iaf_omip2_cycle2', #"01deg_jra55v13_ryf9091",
    "variable": "aice_m",
    "start_time": "2090-02-01",
    "end_time": "2100-01-01",
    "decode_coords": False
}

cat_subset = catalog[sic_args['expt']]
var_search = cat_subset.search(variable=sic_args['variable'])
darray = var_search.to_dask(decode_coords = sic_args['decode_coords'])
darray = darray[sic_args['variable']]
darray = darray.sel(time=slice(sic_args['start_time'], sic_args['end_time']))

sic = darray

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


In [4]:
sic

<xarray.DataArray 'aice_m' (time: 0, nj: 300, ni: 360)>
dask.array<getitem, shape=(0, 300, 360), dtype=float32, chunksize=(0, 150, 180), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 
    TLON     (nj, ni) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    TLAT     (nj, ni) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULON     (nj, ni) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULAT     (nj, ni) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
Dimensions without coordinates: nj, ni
Attributes:
    units:          1
    long_name:      ice area  (aggregate)
    cell_measures:  area: tarea
    cell_methods:   time: mean
    time_rep:       averaged

**Another messy thing about CICE is that it thinks that monthly data for, say, January occurs at midnight on Jan 31 -- while xarray interprets this as the first milllisecond of February.**  
  
To get around this, note that we loaded data from February above, and we now subtract 12 hours from the time dimension. This means that, at least data is sitting in the correct month, and really helps to compute monthly climatologies correctly.

In [5]:
sic['time'] = sic.time.to_pandas() - timedelta(hours = 12)

Note that `aice_m` is the monthly average of fractional ice area in each grid cell aka the concentration. **To find the actual area of the ice we need to know the area of each cell. Unfortunately, CICE doesn't save this for us ... but the ocean model does.** So, let's load `area_t` from the ocean model, and rename the coordinates in our ice variable to match the ocean model. Then we can multiply the ice concentration with the cell area to get a total ice area.

In [6]:
cat_subset = catalog[sic_args['expt']]
var_search = cat_subset.search(variable='area_t')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['area_t']
area_t = darray.load()
area_t

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


<xarray.DataArray 'area_t' (yt_ocean: 300, xt_ocean: 360)>
array([[           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [3.85042360e+07, 1.11973360e+08, 1.80973936e+08, ...,
        1.80973936e+08, 1.11973360e+08, 3.85042360e+07],
       [3.85466840e+07, 1.12088160e+08, 1.81145952e+08, ...,
        1.81145952e+08, 1.12088160e+08, 3.85466840e+07],
       [3.85679440e+07, 1.12145656e+08, 1.81232096e+08, ...,
        1.81232096e+08, 1.12145656e+08, 3.85679440e+07]], dtype=float32)
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
    geolat_t  (yt_ocean, xt_ocean) float32 nan nan nan nan ... 66.04 65.63 65.21
    geolon_t  (yt_ocean, xt_ocean) float32 nan nan nan nan ... 79.96 79.97 79.99
Attributes:
    long_name:     tracer cell area
    units:         m^2
    valid_range:   [0.e+00 1.e+15]
    cell_methods:  time: point

Our CICE data is missing x&y coordinate values, so we can also get them from area_t

In [7]:
sic.coords['ni'] = area_t['xt_ocean'].values
sic.coords['nj'] = area_t['yt_ocean'].values

So that our new coordinates are recognised as cf standard, we also need to copy the attributes. This notebook is designed to use `cf-xarray`. This means the rest of the notebook is [Model Agnostic](https://cosima-recipes.readthedocs.io/en/latest/Tutorials/Model_Agnostic_Analysis.html).

In [8]:
sic.ni.attrs = area_t.xt_ocean.attrs
sic.nj.attrs = area_t.yt_ocean.attrs

In [9]:
sic = sic.rename(({'ni': 'xt_ocean', 'nj': 'yt_ocean'}))

In [10]:
sic.cf

Coordinates:
             CF Axes: * X: ['xt_ocean']
                      * Y: ['yt_ocean']
                        Z, T: n/a

      CF Coordinates:   longitude: ['TLON', 'xt_ocean']
                        latitude: ['TLAT', 'yt_ocean']
                        vertical, time: n/a

       Cell Measures:   area, volume: n/a

      Standard Names:   n/a

              Bounds:   n/a

       Grid Mappings:   n/a

Now that we have axes with cf compliant coordinates, we can select using `latitude` keywords.

Let's look at a timeseries of SH sea ice area. Area is defined (per convention) as the sum of sea ice concentration multiply by the area of each grid cell (and masked for sea ice concentration above 15%)

In [11]:
sic

<xarray.DataArray 'aice_m' (time: 0, yt_ocean: 300, xt_ocean: 360)>
dask.array<getitem, shape=(0, 300, 360), dtype=float32, chunksize=(0, 150, 180), chunktype=numpy.ndarray>
Coordinates:
    TLON      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    TLAT      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULON      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
    ULAT      (yt_ocean, xt_ocean) float32 dask.array<chunksize=(150, 180), meta=np.ndarray>
  * time      (time) datetime64[ns] 
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
Attributes:
    units:          1
    long_name:      ice area  (aggregate)
    cell_measures:  area: tarea
    cell_methods:   time: mean
    time_rep:       averaged

In [12]:
area_t

<xarray.DataArray 'area_t' (yt_ocean: 300, xt_ocean: 360)>
array([[           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [3.85042360e+07, 1.11973360e+08, 1.80973936e+08, ...,
        1.80973936e+08, 1.11973360e+08, 3.85042360e+07],
       [3.85466840e+07, 1.12088160e+08, 1.81145952e+08, ...,
        1.81145952e+08, 1.12088160e+08, 3.85466840e+07],
       [3.85679440e+07, 1.12145656e+08, 1.81232096e+08, ...,
        1.81232096e+08, 1.12145656e+08, 3.85679440e+07]], dtype=float32)
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
    geolat_t  (yt_ocean, xt_ocean) float32 nan nan nan nan ... 66.04 65.63 65.21
    geolon_t  (yt_ocean, xt_ocean) float32 nan nan nan nan ... 79.96 79.97 79.99
Attributes:
    long_name:     tracer cell area
    units:         m^2
    valid_range:   [0.e+00 1.e+15]
    cell_methods:  time: point

In [13]:
(sic * area_t).cf

Coordinates:
             CF Axes: * X: ['xt_ocean']
                      * Y: ['yt_ocean']
                        Z, T: n/a

      CF Coordinates:   longitude: ['TLON', 'ULON', 'geolon_t', 'xt_ocean']
                        latitude: ['TLAT', 'ULAT', 'geolat_t', 'yt_ocean']
                        vertical, time: n/a

       Cell Measures:   area, volume: n/a

      Standard Names:   n/a

              Bounds:   n/a

       Grid Mappings:   n/a

By convention, sea-ice area for a region or basin is the sum of the area's where concentration is greater than 15%. 
We also need to drop geolon and geolat so we have unique longitude and latitude to reference

In [14]:
sic=sic.where(sic >= 0.15)

si_area = (sic * area_t).drop({'geolon_t', 'geolat_t'})

In [15]:
si_area.cf

Coordinates:
             CF Axes: * X: ['xt_ocean']
                      * Y: ['yt_ocean']
                        Z, T: n/a

      CF Coordinates:   longitude: ['TLON', 'ULON', 'xt_ocean']
                        latitude: ['TLAT', 'ULAT', 'yt_ocean']
                        vertical, time: n/a

       Cell Measures:   area, volume: n/a

      Standard Names:   n/a

              Bounds:   n/a

       Grid Mappings:   n/a

In [16]:
SH_area = si_area.cf.sel(latitude=slice(-90, -45)).cf.sum(['latitude', 'longitude'])
NH_area = si_area.cf.sel(latitude=slice(45, 90)).cf.sum(['latitude', 'longitude'])

KeyError: "no index found for coordinate 'ULAT'"

As we are using a repeat year forcing experiemnt, the sea ice cycle is very regular:

In [ ]:
SH_area.plot()
NH_area.plot()
plt.ylabel('Sea Ice Area (km$^{2}$)');

The seasonal cycle of sea-ice concentration is more interesting

In [ ]:
SH_area.groupby('time.month').mean('time').plot(label='Antarctic')
NH_area.groupby('time.month').mean('time').plot(label='Arctic')
plt.legend()
plt.ylabel('Sea Ice Area (km$^{2}$)');

# Making Maps

If we just plot a selected month now, you see that everything North of 65N is skewed. 

In [ ]:
ax=plt.subplot(projection=ccrs.PlateCarree())
sic.sel(time='2095-08-31').plot(transform=ccrs.PlateCarree())
ax.coastlines();

Most of our work is in the Southern Ocean, so maybe we don't care. But if you are interested in the Arctic, then we need to account for the tri-polar ocean grid that ACCESS uses. The easiest way out of that is using contourf, and the passing the x and y coordinates.

See [Making Maps with Cartopy](https://cosima-recipes.readthedocs.io/en/latest/Tutorials/Making_Maps_with_Cartopy.html) tutorial for more help with plotting!

We need the geolon and geolat fields from area_t

In [ ]:
sic=sic.assign_coords({
    'geolat_t': area_t.geolat_t,
    'geolon_t': area_t.geolon_t
})

Use contourf, and the geolon and geolat fields

In [ ]:
ax = plt.subplot(projection=ccrs.PlateCarree())

sic.sel(time='2095-08-31')[0].plot.contourf(transform=ccrs.PlateCarree(),
                                            x='geolon_t', y='geolat_t', levels=33)
ax.coastlines();

Using cartopy, we can make Polar Stereographic plots of sea ice concentration for a selected month, as follows:

In [ ]:
def plot_si_conc(data):
    """ A function for plotting tri-polar data"""

    data.plot.contourf(
        transform=ccrs.PlateCarree(),
        x = 'geolon_t', 
        y = 'geolat_t', 
        levels = 33 ,
        cbar_kwargs = {
            'label':'Sea Ice Concentration'
        }
    )

    ax = plt.gca()

    gl = ax.gridlines(
            draw_labels=True, linewidth=1, color='gray', alpha=0.2, linestyle='--',
            # xlocs=[-150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180],
            # ylocs=[-50, -60, -70, -80]
        )

    ax.coastlines()

In [ ]:
def plot_sh_si_conc():
    ax = plt.subplot(projection=ccrs.SouthPolarStereo())

    plot_si_conc(
        sic.cf.sel(latitude=slice(-90,-45), time='2095-08')[0]
    )

plot_sh_si_conc()

In [ ]:
def plot_nh_si_conc():
    ax = plt.subplot(projection=ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70))

    plot_si_conc(
        sic.cf.sel(latitude=slice(45, 90), time='2095-02')[0]
    )

plot_nh_si_conc()

Once we are happy with your plot, we can save the plot to disk using `plt.savefig('filepath/filename')` function at the end of the cell containing the plot we want to save, as shown below. Note that your filename must contain the file (e.g., pdf, jpeg, png, etc.).  
For more information on the options available to save figures refer to [Matplotlib documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html).

In [ ]:
plot_sh_si_conc()
plt.savefig('MyFirstSeaIcePlot.png', dpi = 300)